In [1]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("local")
         .appName("arrival")
         # Add postgres jar
         .config("spark.driver.extraClassPath", "/home/jovyan/work/jars/postgresql-9.4.1207.jar")
         .getOrCreate())
sc = spark.sparkContext

22/09/29 10:25:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from glob import glob
import pandas as pd

In [14]:
df = pd.read_csv('/home/jovyan/work/data/arrival/m05/links/link_6.csv')
df[df['link_name']=='0_1'][(df['link_timestamp'] > '2022-05-05') & (df['link_timestamp'] < '2022-05-06')][df['path']=='go']\
    .sort_values('link_timestamp').head(20)

/tmp/ipykernel_52/2254228918.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['link_name']=='0_1'][(df['link_timestamp'] > '2022-05-05') & (df['link_timestamp'] < '2022-05-06')][df['path']=='go']\


,link_name,link_time,link_timestamp,path,gps_imei,routes
32149,0_1,525,2022-05-05 05:01:51,go,866381052370738,6
32177,0_1,375,2022-05-05 05:18:28,go,866381052399851,6
32217,0_1,180,2022-05-05 05:31:49,go,866381052396675,6
32273,0_1,120,2022-05-05 05:46:34,go,866381052397913,6
32326,0_1,75,2022-05-05 06:00:56,go,866381052397178,6
32405,0_1,105,2022-05-05 06:16:35,go,866381052358725,6
32505,0_1,135,2022-05-05 06:32:04,go,868998030634154,6
32580,0_1,60,2022-05-05 06:42:14,go,866381052384499,6
32649,0_1,120,2022-05-05 06:51:02,go,866381052396311,6
32738,0_1,150,2022-05-05 07:02:42,go,864507039676258,6


In [9]:
files = glob('/home/jovyan/work/data/old/links/*csv')
files

['/home/jovyan/work/data/old/links/link_6.csv',
 '/home/jovyan/work/data/old/links/link_133.csv',
 '/home/jovyan/work/data/old/links/link_56.csv',
 '/home/jovyan/work/data/old/links/link_35.csv']

In [5]:
df = (spark.read.format('csv')
              .option("header", True)
             .load("/home/jovyan/work/data/old/links/*.csv"))
df.printSchema()

root
 |-- link_name: string (nullable = true)
 |-- link_time: string (nullable = true)
 |-- link_timestamp: string (nullable = true)
 |-- path: string (nullable = true)
 |-- gps_imei: string (nullable = true)
 |-- routes: string (nullable = true)



In [6]:
df = (df.withColumn("link_time",col("link_time(sec)").cast('integer'))
              .withColumn("routes", col("routes").cast('integer'))
             ).select("link_name","link_time","link_timestamp","path","routes")
df.printSchema()

AnalysisException: cannot resolve '`link_time(sec)`' given input columns: [gps_imei, link_name, link_time, link_timestamp, path, routes];
'Project [link_name#16, cast('link_time(sec) as int) AS link_time#28, link_timestamp#18, path#19, gps_imei#20, routes#21]
+- Relation [link_name#16,link_time#17,link_timestamp#18,path#19,gps_imei#20,routes#21] csv


# Upload to postgresql

In [64]:
(df.write
 .format("jdbc")
 .option("url", "jdbc:postgresql://postgres/test")
 .option("dbtable", "public.link_time")
 .option("user", "test")
 .option("password", "postgres")
 .mode("overwrite") # to append change to mode("append")
 .save())